In [1]:
import pandas as pd
# Use a pipeline as a high-level helper
from datasets import Dataset
import torch
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
import json
import os


/Users/ethanyuxin/anaconda3/envs/ai/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
hf_token = os.getenv('HUGGINGFACE_API_KEY')
model_id = "meta-llama/Llama-3.2-1B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    token=hf_token
)

In [8]:
messages = [
    {"role": "user", "content": "Who are you?"},
]
# local model in variable model
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto",
    max_new_tokens=2048,
    temperature=0.7,
    top_p=0.9,
)
pipe(messages)

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


[{'generated_text': [{'role': 'user', 'content': 'Who are you?'},
   {'role': 'assistant',
    'content': 'I\'m an artificial intelligence model known as Llama. Llama stands for "Large Language Model Meta AI."'}]}]

In [15]:
def call_pipe(pipe, section_name, content):
    messages = [{"role": "system", "content": 
                 "You are an AI assistant who will assess whether a given text is used as a background, result or method section in a scientific paper."
                 "You will be given a section name and the text, and you will need to classify the text as one of [background, result, method]."
                 "You will also need to provide a reasoning for your classification."
                 "The output should strictly be a json with the following two keys: classification, reasoning."
                #  "Example output would look like: {\"classification\": \"result\", \"reasoning\": \"This is the reasoning\"}"
    }, 
    {"role": "user", "content": f"section name: {section_name}, text: {content}"}]
    response = pipe(messages)[0]
    try:
        if response["generated_text"][2]["content"]:
            print(response["generated_text"][2]["content"])
            content = response["generated_text"][2]["content"]
            start = content.find("{")
            end = content.rfind("}") + 1
            json_str = content[start:end]
            answers = json.loads(json_str)
            classification = answers["classification"]
            reasoning = answers["reasoning"]
            return classification, reasoning
        else:
            return "Invalid", "No reasoning provided"
    except Exception as e:
        print(e)
        return "Invalid", "No reasoning provided"

In [16]:
model_classification, model_reasoning = call_pipe(pipe, "Introduction", "However, how frataxin interacts with the Fe-S cluster biosynthesis components remains unclear as direct one-to-one interactions with each component were reported (IscS [12,22], IscU/Isu1 [6,11,16] or ISD11/Isd11 [14,15]).")

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


{
  "classification": "method",
  "reasoning": "This section is describing a method or experimental approach used to investigate the interaction between frataxin and the Fe-S cluster biosynthesis components, but it does not report the results of that investigation. It appears to be a background section that provides context and context about the method used to study the interaction."
}


In [17]:
print(f"{model_classification}")
print(f"{model_reasoning}")

method
This section is describing a method or experimental approach used to investigate the interaction between frataxin and the Fe-S cluster biosynthesis components, but it does not report the results of that investigation. It appears to be a background section that provides context and context about the method used to study the interaction.


In [ ]:

# def preprocess(file_path):
#     data = pd.read_json(path_or_buf=file_path, lines=True)
#     ids = set()
#     rows_to_be_dropped = []
#     for i in range(len(data)):
#         row = data.iloc[i]
#         if row.unique_id in ids:
#             rows_to_be_dropped.append(i)
#         else:
#             ids.add(row.unique_id)
#     data = data.drop(rows_to_be_dropped)
#     return data

In [ ]:
# data = preprocess("./data/train.jsonl")

In [ ]:
# labels = []
# reasonings = []
# raw_output = []
# ids = []

# for i in range(len(data)):
#     current_data = data.iloc[i]
#     ids.append(current_data.unique_id)
#     classification, reasoning = call_pipe(pipe, current_data.sectionName, current_data.string)
#     raw_output.append(classification)
#     labels.append(classification)
#     reasonings.append(reasoning)

In [ ]:
# df = pd.DataFrame(zip(ids, labels, reasonings), columns=["id", "model_classification", "reasoning"])
# df.to_csv("llama-1b.csv")